# Vehicle Sales Data Analysis
This notebook outlines the code and results of the ten data analysis tasks for the BS3220 Parallel Programming assignment. 


### Import libraries and load data

In [177]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, count, max, min, avg, col, rank, to_date, date_format, regexp_extract, row_number, first, last, format_number, substring
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder \
    .appName("VehicleSalesCleaning") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

df = spark.read.csv("car_prices.csv", header=True, inferSchema=True)


### Data cleaning

In [178]:
# Display the original data (optional, for verification)
print("Original Data:")
df.show(5)

df = df.filter(df.sellingprice != 1)

# Extract the date part excluding time and timezone using regular expressions
df = df.withColumn("date_part", regexp_extract(col("saledate"), r"(\w+\s\w+\s\d+\s\d+)", 1))

# Display the extracted date part (optional, for verification)
print("Extracted Date Part:")
df.select("saledate", "date_part").show(5)

# Convert extracted string to date and format it to MMddyyyy
df = df.withColumn("formatted_saledate", date_format(to_date(col("date_part"), "EEE MMM dd yyyy"), "MMddyyyy"))

# Show the final transformed data to verify the transformation
print("Transformed Data:")
df.select("saledate", "formatted_saledate").show(5)

Original Data:
+----+-----+-------------------+----------+-----+------------+-----------------+-----+---------+--------+-----+--------+--------------------+-----+------------+--------------------+
|year| make|              model|      trim| body|transmission|              vin|state|condition|odometer|color|interior|              seller|  mmr|sellingprice|            saledate|
+----+-----+-------------------+----------+-----+------------+-----------------+-----+---------+--------+-----+--------+--------------------+-----+------------+--------------------+
|2015|  Kia|            Sorento|        LX|  SUV|   automatic|5xyktca69fg566472|   ca|        5|   16639|white|   black|kia motors americ...|20500|       21500|Tue Dec 16 2014 1...|
|2015|  Kia|            Sorento|        LX|  SUV|   automatic|5xyktca69fg561319|   ca|        5|    9393|white|   beige|kia motors americ...|20800|       21500|Tue Dec 16 2014 1...|
|2014|  BMW|           3 Series|328i SULEV|Sedan|   automatic|wba3c1c51ek11

### Task 1: Find the total sales for each item, both the number of units and the total price/cost

In [179]:
# from pyspark.sql.functions import sum, count

# df.groupBy("make", "model").agg(
#     count("vin").alias("units_sold"), 
#     sum("sellingprice").alias("total_revenue") 
# ).show(20)



### Task 2: Summarise the total sales of all items at each location

In [180]:
# salesPerState = df.groupBy("state").agg(
#     sum("sellingprice").alias("total_revenue")
# ).orderBy("total_revenue", ascending=False)

# salesPerState.show(55)

### Task 3: List all products and their combined sales, grouped by their location of sale.

In [181]:
#TODO - Change

combined_sales_by_state_and_product = df.groupBy("state", "make", "model").agg(
    sum("sellingprice").alias("total_revenue")
).orderBy("total_revenue", ascending=False)

# Display the combined sales for each product by state
combined_sales_by_state_and_product.show(30)

+-----+-------------+--------------+-------------+
|state|         make|         model|total_revenue|
+-----+-------------+--------------+-------------+
|   fl|       Nissan|        Altima|     40266403|
|   ca|          BMW|      3 Series|     39475195|
|   ca|     Infiniti|       G Sedan|     37101104|
|   tx|         Ford|         F-150|     35473601|
|   fl|       Toyota|         Camry|     33998428|
|   ca|       Nissan|        Altima|     31781931|
|   mi|         Ford|         F-150|     31000235|
|   pa|       Nissan|        Altima|     27048715|
|   ca|          BMW|      5 Series|     26879650|
|   pa|         Ford|         F-150|     24072500|
|   ca|Mercedes-Benz|       E-Class|     22858804|
|   fl|        Lexus|        RX 350|     22784950|
|   pa|        Honda|        Accord|     22761300|
|   fl|     Infiniti|       G Sedan|     22361802|
|   tn|         Ford|         F-150|     22239125|
|   ca|Mercedes-Benz|       C-Class|     21891527|
|   fl|       Toyota|       Cor

### Task 4: Show the sales numbers for the item which sold the most units at each location

In [182]:
# item_sales_by_state = df.groupBy("state", "make", "model", "trim").agg(
#     count("vin").alias("units_sold"),
#     sum("sellingprice").alias("total_revenue")
# )

# # Define a window partitioned by state and ordered by units_sold in descending order
# windowSpec = Window.partitionBy("state").orderBy(col("units_sold").desc())

# # Rank items within each state and select the top item
# top_item_by_state = item_sales_by_state.withColumn("rank", rank().over(windowSpec)).filter(col("rank") == 1)
# top_item_by_state.show(400)

### Task 5: List all items that were sold within two months of your choosing

In [188]:
#TODO - change

df_filtered = df.filter((col("formatted_saledate").substr(1,2) == "01") | (col("formatted_saledate").substr(1,2)=="02"))

df_filtered.select("make", "model", "saledate", "formatted_saledate").show()


+---------+--------+--------------------+------------------+
|     make|   model|            saledate|formatted_saledate|
+---------+--------+--------------------+------------------+
|      BMW|3 Series|Thu Jan 15 2015 0...|          01152015|
|    Volvo|     S60|Thu Jan 29 2015 0...|          01292015|
|Chevrolet|  Camaro|Tue Jan 20 2015 0...|          01202015|
|     Ford|  Fusion|Tue Jan 13 2015 1...|          01132015|
|      BMW|5 Series|Tue Feb 03 2015 0...|          02032015|
|      BMW|6 Series|Tue Jan 06 2015 1...|          01062015|
|    Volvo|    XC70|Thu Feb 26 2015 0...|          02262015|
|    Volvo|    XC70|Thu Feb 12 2015 0...|          02122015|
|     Audi|     SQ5|Thu Jan 29 2015 0...|          01292015|
|    Buick|  Verano|Tue Jan 06 2015 0...|          01062015|
|      BMW|3 Series|Thu Jan 15 2015 0...|          01152015|
|      BMW|      M5|Tue Jan 13 2015 0...|          01132015|
|      BMW|3 Series|Thu Jan 15 2015 0...|          01152015|
| Cadillac|     ELR|Wed 

### Task 6: Identify the item which has the lowest overall sales, both for the dataset as a whole and for each sales location

In [184]:
# salesPerCar = df.groupBy("make", "model").agg(
#     count("*").alias("unitsSold")
# )
# salesPerCarPerState = df.groupBy("state", "make", "model").agg(
#     count("*").alias("unitsSold")
# )
# lowestCarSale = salesPerCar.orderBy("unitsSold").first()

# windowSpec = Window.partitionBy("state").orderBy("unitsSold")
# lowestCarSaleByState = salesPerCarPerState.withColumn("row_number", row_number().over(windowSpec)).filter(col("row_number") == 1)

# print("Lowest selling car:")
# print(lowestCarSale)
# print("Lowest Selling Car by State:")
# lowestCarSaleByState.show(55)

### Task 7: Find the most expensive and least expensive item for each location where sales occurred

In [185]:
# windowExpensive = Window.partitionBy("state").orderBy(col("sellingprice").desc())
# windowCheap = Window.partitionBy("state").orderBy(col("sellingprice").asc())

# dfRanked = df.withColumn("rank_desc", row_number().over(windowExpensive)) \
#              .withColumn("rank_asc", row_number().over(windowCheap))

# mostExpensive = dfRanked.filter(col("rank_desc") == 1).select(
#     "state", "make", "model", "sellingprice"
# )
# leastExpensive = dfRanked.filter(col("rank_asc") == 1).select(
#     "state", "make", "model", "sellingprice"
# )

# print("Most Expensive Car Sale by State:")
# mostExpensive.show(10)
# print("Least Expensive Car Sale by State:")
# leastExpensive.show(10)




### Task 8: Calculate the average cost of an item at each location within your dataset

In [186]:
# averagePriceByState = df.groupBy("state").agg(
#     format_number(avg("sellingprice"), 2).alias("average_selling_price")
# )
# averagePriceByState = averagePriceByState.orderBy("state")

# print("Average Selling Price by State:")
# averagePriceByState.show(55)

### Task 9: Based on your individual dataset, create a set of variables which can be used as broadcast variables.

### Task 10: Complete one other query to analyse the data, based on your individual dataset. This query should be relevant to your own specific dataset.

In [187]:
# Query: to show the best performing months across all states and all dealerships


stateSalesAndRevenue = df.groupBy("state").agg(count("*").alias("numberOfSales"), sum("sellingprice").alias("totalRevenue"))

stateSales = df.groupBy("state").agg(count("*").alias("numberOfSales"))
stateRevenue = df.groupBy("state").agg(sum("sellingprice").alias("totalRevenue"))
stateAverage = df.groupBy("state").agg(avg("sellingprice").alias("averageRevenuePerSale"))

bestStateForSales = stateSales.orderBy(col("numberOfSales").desc())
bestStateForRevenue = stateRevenue.orderBy(col("totalRevenue").desc())
bestStateForAverageRevenuePerSale = stateAverage.orderBy(col("averageRevenuePerSale").desc())

#Formatting 2DP for revenue after ordering as was causing table to sort lowest first, not highest first 
formattedBestStateForAverageRevenue = bestStateForAverageRevenuePerSale.withColumn(
    "formattedAverageRevenue", format_number(col("averageRevenuePerSale"), 2)
)

print("Best performing state for sales:")
bestStateForSales.show(10)
print("Best performing state for revenue:")
bestStateForRevenue.show(10)
print("Best average revenue per sale:")
formattedBestStateForAverageRevenue.select("state", "formattedAverageRevenue").show(10)

Best performing state for sales:
+-----+-------------+
|state|numberOfSales|
+-----+-------------+
|   fl|        82945|
|   ca|        73147|
|   pa|        53907|
|   tx|        45912|
|   ga|        34749|
|   nj|        27784|
|   il|        23478|
|   nc|        21845|
|   oh|        21575|
|   tn|        20895|
+-----+-------------+
only showing top 10 rows

Best performing state for revenue:
+-----+------------+
|state|totalRevenue|
+-----+------------+
|   fl|  1151064556|
|   ca|  1061077776|
|   pa|   861234573|
|   tx|   606495360|
|   ga|   448927413|
|   nj|   378115350|
|   tn|   355418608|
|   il|   347290026|
|   oh|   310822810|
|   mo|   231647138|
+-----+------------+
only showing top 10 rows

Best average revenue per sale:
+-----+-----------------------+
|state|formattedAverageRevenue|
+-----+-----------------------+
|   on|              17,812.62|
|   tn|              17,009.74|
|   pa|              15,976.30|
|   co|              15,877.80|
|   nv|              15